In [1]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("tohoku-nlp/bert-base-japanese-whole-word-masking")
model = AutoModel.from_pretrained("tohoku-nlp/bert-base-japanese-whole-word-masking")

/home/thomas/miniconda3/envs/geonlp/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/thomas/miniconda3/envs/geonlp/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [1]:
entity = "東大"

In [19]:
import torch


def get_entity_embedding(entity: str):
    input = tokenizer(entity, return_tensors="pt")
    with torch.no_grad():
        output = model(**input)
    embeddings = output.last_hidden_state
    entity_embedding = embeddings.mean(dim=1)
    return entity_embedding

In [20]:
entity_embedding = get_entity_embedding(entity)

In [21]:
tokyo_embedding = get_entity_embedding("東京")
kyoto_embedding = get_entity_embedding("京都")
nara_embedding = get_entity_embedding("奈良")

In [22]:
from sklearn.metrics.pairwise import cosine_similarity

print(cosine_similarity(entity_embedding, kyoto_embedding))
print(cosine_similarity(entity_embedding, nara_embedding))
print(cosine_similarity(entity_embedding, tokyo_embedding))

[[0.7894262]]
[[0.8048662]]
[[0.8820897]]


# Filter nodes without name tag

In [25]:
len(tokyo_embedding[0])

768

In [1]:
# periodically execute sudo sync; echo 1 > /proc/sys/vm/drop_caches

from lxml import etree

def filter_nodes(input_file, output_file):
    with open(output_file, 'w', encoding='utf-8') as output:
        output.write("<?xml version='1.0' encoding='UTF-8'?>\n")
        output.write("<osm version=\"0.6\" generator=\"osmium/1.14.0\">\n")

        context = etree.iterparse(input_file, events=('start', 'end'))
        _, root = next(context)  # Get the root element
        i = 0
        for event, elem in context:
            i += 1
            if i % 1_000_000 == 0:
                print(i // 1_000_000)
            if event == 'start' and elem.tag == 'node':
                name_tag = elem.xpath("tag[@k='name']")
                if name_tag:
                    output.write(etree.tostring(elem, encoding='unicode'))
            elem.clear()
            while elem.getprevious() is not None:
                del elem.getparent()[0]
        print(i)
        output.write("</osm>")


In [2]:
# Usage
input_file = "../data/japan-latest.osm"
output_file = "../data/nodes_with_names.osm"
filter_nodes(input_file, output_file)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


# Count nodes in the filtered file

In [4]:
# Count nodes in the filtered file
def count_nodes(input_file):
    context = etree.iterparse(input_file, events=('start', 'end'))
    _, root = next(context)  # Get the root element
    i = 0
    for event, elem in context:
        if event == 'start' and elem.tag == 'node':
            i += 1
        elem.clear()
        while elem.getprevious() is not None:
            del elem.getparent()[0]
    print(i)

In [6]:
count_nodes(output_file)

1265618


# Filter duplicate nodes

In [9]:
import xml.etree.ElementTree as ET

def create_id_string(node):
    allowed_tags = [
        "prefecture", "city", "suburb", "quarter", "neighbour", "road",
        "aerialway", "aeroway", "amenity", "artwork_type", "attraction",
        "branch", "building", "craft", "castle_type", "highway", "historic",
        "leisure", "man_made", "museum", "natural", "office", "place",
        "public_transport", "railway", "route", "route_master", "ruins",
        "shop", "tourism", "tower:type", "water", "waterway", "name"
    ]

    tags = []
    for tag in node.findall('tag'):
        key = tag.attrib.get('k', '')
        if key in allowed_tags:
            value = tag.attrib.get('v', '')
            tags.append(f"{key}={value.replace('&#x', '').replace(';', ' ').strip()}")

    return '|'.join(tags)

def filter_duplicates(input_file, output_file):
    tree = ET.parse(input_file)
    root = tree.getroot()
    id_strings = {}
    filtered_nodes = []
    i = 0
    for node in root.findall('node'):
        i += 1
        if i==10_000:
            break
        if i % 100_000 == 0:
            print(i // 100_000)
        id_string = create_id_string(node)
        if id_string not in id_strings:
            filtered_nodes.append(node)
            id_strings[id_string] = node
        else:
            if "highway" not in id_string and "railway" not in id_string:
                print(id_string)
                print(node.attrib)
                for child in node:
                    print(child.tag, child.attrib)
                print("----")
                print(id_strings[id_string].attrib)
                for child in id_strings[id_string]:
                    print(child.tag, child.attrib)
    i = 0
    # Write filtered nodes to the output file
    # with open(output_file, 'w', encoding='utf-8') as f:
    #     f.write('<?xml version=\'1.0\' encoding=\'UTF-8\'?>\n')
    #     f.write('<osm version="0.6" generator="osmium/1.14.0">\n')
    #     for node in filtered_nodes:
    #         i += 1
    #         if i % 100_000 == 0:
    #             print(i // 100_000)
    #         f.write(ET.tostring(node, encoding='unicode'))
    #     f.write('</osm>')
    # print(f"Filtered nodes: {len(filtered_nodes)}")


In [10]:
filter_duplicates("../data/nodes_with_names.osm", "../data/nodes_with_names_no_duplicates.osm")

name=銀座七丁目
{'id': '31254332', 'version': '7', 'timestamp': '2022-08-30T22:26:39Z', 'lat': '35.6685462', 'lon': '139.7621899'}
tag {'k': 'junction', 'v': 'yes'}
tag {'k': 'name', 'v': '銀座七丁目'}
tag {'k': 'name:en', 'v': 'Ginza 7'}
tag {'k': 'name:ja', 'v': '銀座七丁目'}
----
{'id': '31254325', 'version': '8', 'timestamp': '2023-11-15T03:30:28Z', 'lat': '35.66851', 'lon': '139.7622349'}
tag {'k': 'junction', 'v': 'yes'}
tag {'k': 'name', 'v': '銀座七丁目'}
tag {'k': 'name:en', 'v': 'Ginza 7'}
tag {'k': 'name:ja', 'v': '銀座七丁目'}
tag {'k': 'name:ko', 'v': '긴자7초메'}
name=芝四丁目
{'id': '31295408', 'version': '8', 'timestamp': '2022-08-31T22:39:53Z', 'lat': '35.6491821', 'lon': '139.7530113'}
tag {'k': 'junction', 'v': 'yes'}
tag {'k': 'name', 'v': '芝四丁目'}
tag {'k': 'name:en', 'v': 'Shiba 4-chome'}
tag {'k': 'name:ja', 'v': '芝四丁目'}
----
{'id': '31295407', 'version': '8', 'timestamp': '2022-08-31T22:39:53Z', 'lat': '35.649136', 'lon': '139.7532039'}
tag {'k': 'junction', 'v': 'yes'}
tag {'k': 'name', 'v': '芝

In [22]:
947179/1265618

0.748392484936213

In [23]:
1.8/2.8

0.6428571428571429

In [24]:
count_nodes("../data/node_with_names_no_duplicates.osm")

947179
